# Assignment 3: Policy Gradients (DDPG and REINFORCE)

Name: Nutcha Temiyasathit

ID: A53249944

## Background
This exercise requires you to solve various continous control problems in OpenAI-Gym.  

DDPG is policy gradient actor critic method for continous control which is off policy. It tackles the curse of dimensionality / loss of performance faced when discretizing a continous action domain. DDPG uses similiar "tricks" as DQN to improve the stability of training, including a replay buffer and target networks.

Furthermore, you will implement REINFORCE for discrete and continous environments, and as a bonus compare the sample efficiency and performance with DQN and DDPG.


### DDPG paper: https://arxiv.org/pdf/1509.02971.pdf

### Environments:

#### InvertedPendulum-v2 environment:
<img src="inverted_pendulum.png" width="300">

#### Pendulum-v0 environment:
<img src="pendulum.png" width="300">

#### Halfcheetah-v2 environment:
<img src="half_cheetah.png" width="300">



### Setup environment for Actor Critic
- inline plotting
- gym
- directory for logging videos

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import random

#environment
import gym

#pytorch
import torch
from torch.autograd import Variable
import torch.nn as nn

logging_interval = 100
animate_interval = logging_interval * 5
logdir='./DDPG/'

### Set up gym environment
The code below does the following for you:
- Wrap environment, log videos, setup CUDA variables (if GPU is available)
- Record action and observation space dimensions
- Fix random seed for determinisitic training

In [2]:
VISUALIZE = False
SEED = 0
MAX_PATH_LENGTH = 500
NUM_EPISODES = 12000
GAMMA=0.99
BATCH_SIZE = 128

# Environments to be tested on
# env_name = 'InvertedPendulum-v1'
env_name = 'Pendulum-v0'
#env_name = 'HalfCheetah-v1' 

# wrap gym to save videos
env = gym.make(env_name)
if VISUALIZE:
    if not os.path.exists(logdir):
        os.mkdir(logdir)
    env = gym.wrappers.Monitor(env, logdir, force=True, video_callable=lambda episode_id: episode_id%logging_interval==0)
env._max_episodes_steps = MAX_PATH_LENGTH

# check observation and action space
discrete = isinstance(env.action_space, gym.spaces.Discrete)
obs_dim = env.observation_space.shape[0]
act_dim = env.action_space.n if discrete else env.action_space.shape[0]

if discrete:
    print("This is a discrete action space, probably not the right algorithm to use")

# set random seeds
torch.manual_seed(SEED)
np.random.seed(SEED)

# make variable types for automatic setting to GPU or CPU, depending on GPU availability
use_cuda = torch.cuda.is_available()
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if use_cuda else torch.ByteTensor
Tensor = FloatTensor

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


### Demonstrate your understanding of the simulation:
For the environments mentioned above ('Pendulum-v0', 'HalfCheetah-v1', 'InvertedPendulum-v1'),
- describe the reward system
- describe the each state variable (observation space)
- describe the action space
- when is the environment considered "solved"?

Ans: 


<b> Inverted Pendulum </b>
- Reward: 
- Observation Space (State):
- Action Space: 
- Solved: 

<b> Pendulum </b>
- Reward: 
- Observation Space (State):
- Action Space: 
- Solved: 

<b> Half Cheetah </b>
- Reward: 
- Observation Space (State):
- Action Space: 
- Solved: 

### Implement an action normalization class:
To train across various environments, it is useful to normalize action inputs and outputs between [-1, 1]. This class should take in actions and implement forward and reverse functions to map actions between [-1, 1] and [action_space.low, action_space.high].

Using the following gym wrapper, implement this class.
- https://github.com/openai/gym/blob/78c416ef7bc829ce55b404b6604641ba0cf47d10/gym/core.py
- i.e. we are overriding the outputs scale of actions. 

In [3]:
class NormalizeAction(gym.ActionWrapper):
    def action(self, action):
        #tanh outputs (-1,1) from tanh, need to be [action_space.low, action_space.high]
        print("this is called")
        action_range = self.action_space.high - self.action_space.low
        rev_action = ((action + 1)*action_range/2 + self.action_space.low)
        return rev_action
        
    def reverse_action(self, action):
        #reverse of that above
        action_range = self.action_space.high - self.action_space.low
        return (2*(action-self.action_space.low)/action_range) - 1

# DDPG

### Write a weight syncing function
In contrast to DQN, DDPG uses soft weight sychronization. At each time step following training, the actor and critic target network weights are updated to track the rollout networks. 
- target_network.weights <= target_network.weights \* (1 - tau) + source_network.weights \* (tau)

In [4]:
def weightSync(target_model, source_model, tau = 0.001):
    for parameter_target, parameter_source in zip(target_model.parameters(), source_model.parameters()):
        parameter_target.data.copy_((1 - tau) * parameter_target.data + tau * parameter_source.data)

### Write a Replay class that includes all the functionality of a replay buffer
DDPG is an off policy actor-critic method and an identical replay buffer to that used for the previous assignment is applicable here as well (do not include the generate_minibatch method in your Replay class this time). Like before, your constructor for Replay should create an initial buffer of size 1000 when you instantiate it.

The replay buffer should kept to some maximum size (60000), allow adding of samples and returning of samples at random from the buffer. Each sample (or experience) is formed as (state, action, reward, next_state, done). 

In [5]:
class Replay:
    def __init__(self, max_size=60000): 
        self.max_size = max_size
        self.memory = []
    
    def push(self, transition): 
        self.memory.append(transition) 
        if(len(self.memory)) > self.max_size:
            del self.memory[0]

    def initialize(self, env, init_length=1000): 
        state = env.reset()
        for i in range(init_length):
            action = np.random.uniform(-1.,1.,env.action_space.shape[0])
            next_state, reward, done, _ = env.step(action)
            transition = (FloatTensor([state]),
                          FloatTensor([action]),
                          FloatTensor([reward]),
                          FloatTensor([next_state]), 
                          done)
            self.push(transition) 
            if done:
                state = env.reset() 
            else:
                state = next_state
                
    def sample_minibatch(self, batch_size):
        batch = random.sample(self.memory, batch_size)
        return zip(*batch)

### Write an Ornstein Uhlenbeck process class for exploration noise
The proccess is described here:
- https://en.wikipedia.org/wiki/Ornstein–Uhlenbeck_process
- http://math.stackexchange.com/questions/1287634/implementing-ornstein-uhlenbeck-in-matlab

You should implement:
- a step / sample method
- reset method

Use theta = 0.15, mu = 0, sigma = 0.3, dt = 0.01

In [6]:
class OrnsteinUhlenbeckProcess():
    
    def __init__(self, dimension, num_steps, mu=0, sigma=0.3, theta=.15, dt=1e-2, x0=None):
        self.dim = dimension
        self.num_steps = num_steps
        self.theta = theta
        self.mu = np.zeros(dimension)
        self.sigma = sigma
        self.dt = dt
        self.x0 = x0
        self.reset()
    
    def step(self):
        noise = np.sqrt(self.dt) * np.random.normal(size=self.mu.shape)
        x = self.x_prev + self.theta * (self.mu - self.x_prev) * self.dt + self.sigma * noise
        self.x_prev = x
        return x
        
    def reset(self):
        self.x_prev = self.x0 if self.x0 is not None else np.zeros_like(self.mu)

### Write a Deep Neural Network class that creates a dense network of a desired architecture for actor and critic networks


#### Actor
- input and hidden layer activation function: ReLU

- output activation function: Tanh

- hidden_state sizes: 400

- state and action sizes: variable

- number of hidden layers: 2

- batch normalization applied to all hidden layers

- weight initialization: normal distribution with small variance. 

#### Critic
- input and hidden layer activation function: ReLU

- output activation function: None

- hidden_state sizes: 300, 300 + action size

- state and action sizes: variable

- number of hidden layers: 2

- batch normalization applied to all hidden layers prior to the action input

- weight initialization: normal distribution with small variance.

Good baselines can be found in the paper.

In [7]:
# ----------------------------------------------------
# actor model, MLP
# ----------------------------------------------------
# 2 hidden layers, 400 units per layer, tanh output to bound outputs between -1 and 1

def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.normal(tensor=m.weight, std=0.02)

class actor(nn.Module):
    
    def __init__(self, input_size, output_size, hidden_size=400):
        super(actor, self).__init__()
        self.input = nn.Linear(input_size, hidden_size)
        self.input_bn = nn.BatchNorm1d(hidden_size)
        self.h1 = nn.Linear(hidden_size, hidden_size)
        self.h1_bn = nn.BatchNorm1d(hidden_size)
        self.h2 = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        x = self.input(x)
        x = self.input_bn(x)
        x = F.relu(x)
        x = self.h1(x)
        x = self.h1_bn(x)
        x = F.relu(x) 
        x = self.h2(x)
        x = F.tanh(x)
        return x
    
# ----------------------------------------------------
# critic model, MLP
# ----------------------------------------------------
# 2 hidden layers, 300 units per layer, ouputs rewards therefore unbounded
# Action not to be included until 2nd layer of critic (from paper). Make sure to formulate your critic.forward() accordingly

class critic(nn.Module):
    
    def __init__(self, state_size, action_size, output_size, hidden_size=300):
        super(critic, self).__init__()
        self.input = nn.Linear(state_size, hidden_size)
        self.input_bn = nn.BatchNorm1d(hidden_size)
        self.h1 = nn.Linear(hidden_size + action_size, hidden_size)
        self.h2 = nn.Linear(hidden_size, output_size)

    def forward(self, s, a):
        s = self.input(s)
        s = self.input_bn(s)
        s = F.relu(s)
        x = torch.cat((s, a), 0)
        x = self.h1(x)
        x = F.relu(x) 
        x = self.h2(x)
        return x

### Define DDPG class to encapsulate definition, rollouts, and training

- gamma = 0.99

- actor_lr = 1e-4

- critic_lr = 1e-3

- critic l2 regularization = 1e-2

- noise decay

- noise class

- batch_size = 128

- optimizer: Adam

- loss (critic): mse

Furthermore, you can experiment with action versus parameter space noise. The standard implimentation works with action space noise, howeve parameter space noise has shown to produce excellent results.


In [8]:
class DDPG:
    def __init__(self, obs_dim, act_dim, critic_lr = 1e-3, actor_lr = 1e-4, gamma = GAMMA, batch_size = BATCH_SIZE):
        
        self.gamma = GAMMA
        self.batch_size = BATCH_SIZE
        
        # actor
        self.actor = actor(input_size = obs_dim, output_size = act_dim).type(FloatTensor)
        self.actor.apply(init_weights)
        self.actor_target = actor(input_size = obs_dim, output_size = act_dim).type(FloatTensor)
        self.actor_target.load_state_dict(self.actor.state_dict())

        # critic
        self.critic = critic(state_size = obs_dim, action_size = act_dim, output_size = 1).type(FloatTensor)
        self.critic.apply(init_weights)
        self.critic_target = critic(state_size = obs_dim, action_size = act_dim, output_size = 1).type(FloatTensor)
        self.critic_target.load_state_dict(self.critic.state_dict())

        # optimizers
        self.optimizer_actor = torch.optim.Adam(self.actor.parameters(), lr = actor_lr)
        self.optimizer_critic = torch.optim.Adam(self.critic.parameters(), lr = critic_lr, weight_decay=1e-2)
        
        # critic loss
        self.critic_loss = nn.MSELoss()
        
        # noise
        self.noise = OrnsteinUhlenbeckProcess(dimension = act_dim, num_steps = NUM_EPISODES)

        # replay buffer 
        self.replayBuffer = Replay()
        self.replayBuffer.initialize(env)
        
        
    def train(self):
     
        # sample from Replay
        batch_s, batch_a, batch_r, batch_ns, batch_d = self.replayBuffer.sample_minibatch(self.batch_size)
        
        batch_state = Variable(torch.cat(batch_s))
        batch_action = Variable(torch.cat(batch_a))
        batch_reward = Variable(torch.cat(batch_r))
        batch_next_state = Variable(torch.cat(batch_ns))

        # update critic (create target for Q function)
        target_critic_action = self.actor_target(batch_next_state)
        not_done_mask = Variable(torch.from_numpy(1-np.array(list(batch_d)))).float()
        target = not_done_mask * self.gamma * self.critic_target(batch_next_state, target_critic_action)
        target = batch_reward + target
        
        # critic optimizer and backprop step (feed in target and predicted values to self.critic_loss)
        prediction = self.critic(batch_state, batch_action)
        self.optimizer_critic.zero_grad()
        loss_critic = self.critic_loss(target, prediction)
        loss_critic.backward()
        self.optimizer_critic.step()
        
        # update actor (formulate the loss wrt which actor is updated)
        predicted_action = self.actor(batch_state)
        loss_actor = -1 * torch.sum(self.critic(batch_state, predicted_action))
        
        # actor optimizer and backprop step (loss_actor.backward())
        self.optimizer_actor.zero_grad()
        loss_actor.backward()
        self.optimizer_actor.step()

        # sychronize target network with fast moving one
        weightSync(self.critic_target, self.critic)
        weightSync(self.actor_target, self.actor)

### Create an instance of your DDPG object
- Print network architectures, confirm they are correct

In [9]:
ddpg = DDPG(obs_dim = obs_dim, act_dim = act_dim)
print(ddpg.actor)
print(ddpg.critic)

actor(
  (input): Linear(in_features=3, out_features=400, bias=True)
  (input_bn): BatchNorm1d(400, eps=1e-05, momentum=0.1, affine=True)
  (h1): Linear(in_features=400, out_features=400, bias=True)
  (h1_bn): BatchNorm1d(400, eps=1e-05, momentum=0.1, affine=True)
  (h2): Linear(in_features=400, out_features=1, bias=True)
)
critic(
  (input): Linear(in_features=3, out_features=300, bias=True)
  (input_bn): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True)
  (h1): Linear(in_features=301, out_features=300, bias=True)
  (h2): Linear(in_features=300, out_features=1, bias=True)
)


### Train DDPG on different environments
Early stopping conditions:
- avg_val > 500 for "InvertedPendulum" 
- avg_val > -150 for "Pendulum" 
- avg_val > 1500 for "HalfCheetah" 



In [10]:
env = NormalizeAction(env) # remap action values for the environment
avg_val = 0

#for plotting
running_rewards_ddpg = []
step_list_ddpg = []
step_counter = 0

# set term_condition for early stopping according to environment being used
# term_condition = 450 # InvertedPendulum
term_condition = -150 # Pendulum
# term_condition = 1500 # Cheetah

for itr in range(NUM_EPISODES):
    state = env.reset() # get initial state
    done = False
    animate_this_episode = (itr % animate_interval == 0) and VISUALIZE

    while True:
        ddpg.noise.reset()

        if animate_this_episode:
                env.render()
                time.sleep(0.05)

        # use actor to get action, add ddpg.noise.step() to action
        action = ddpg.actor(Variable(FloatTensor([state]), volatile=True))
        action = action.data.numpy().squeeze(0) + ddpg.noise.step()
        action = np.clip(action, -1., 1.)
        
        # remember to put NN in eval mode while testing (to deal with BatchNorm layers) and put it back 
        # to train mode after you're done getting the action
                
        # step action, get next state, reward, done (keep track of total_reward)
        # populate ddpg.replayBuffer
        next_state, reward, done, _ = env.step(action)
        
        ddpg.replayBuffer.push(FloatTensor([state]),
                               FloatTensor([action]),
                               FloatTensor([reward]),
                               FloatTensor([next_state]), 
                               done)
        ddpg.train()
        
        total_reward += reward
        state = next_state
        step_counter += 1
        
        if done:
            break

    if avg_val > term_condition:
        break

    running_rewards_ddpg.append(total_reward) # return of this episode
    step_list_ddpg.append(step_counter)

    avg_val = avg_val * 0.95 + 0.05*running_rewards[-1]
    print("Average value: {} for episode: {}".format(avg_val,itr))


Variable containing:

Columns 0 to 9 
 0.2041  0.5352  0.5851 -0.5189 -0.3550 -0.2376 -0.3821  0.5169 -0.0260 -0.4089

Columns 10 to 19 
-0.2585 -0.3289  0.2200 -0.4592 -0.5264 -0.5537  0.1932  0.3385 -0.3525  0.1070

Columns 20 to 29 
 0.4438  0.0348 -0.1094  0.0306 -0.0526  0.2752 -0.4142  0.3112  0.3235 -0.3808

Columns 30 to 39 
 0.4560  0.5683 -0.2409 -0.1474  0.2803 -0.4296  0.2207  0.1232  0.3368 -0.5012

Columns 40 to 49 
-0.4840  0.2365  0.2352 -0.5211 -0.2859 -0.5213 -0.4679 -0.3100  0.4561 -0.1052

Columns 50 to 59 
 0.3420  0.5542  0.2518 -0.5802  0.2661  0.1958  0.4631 -0.5621  0.3860  0.4919

Columns 60 to 69 
-0.5551  0.0604  0.5482 -0.4266  0.2051  0.1965  0.1873  0.5966  0.4259  0.5271

Columns 70 to 79 
-0.0548  0.4620 -0.5304 -0.3059 -0.1507 -0.1510  0.5558  0.2550  0.3644  0.4180

Columns 80 to 89 
-0.4084 -0.0641  0.1128  0.1669  0.3149 -0.2249 -0.1961  0.4614 -0.1293 -0.3903

Columns 90 to 99 
 0.3768  0.3683 -0.1711 -0.1169 -0.4848 -0.4972  0.2369 -0.2323  0.4466

ValueError: Expected more than 1 value per channel when training, got input size [1, 400]

## Plot rewards over multiple training runs 
This is provided to generate and plot results for you.

In [ ]:
def numpy_ewma_vectorized_v2(data, window):

    alpha = 2 /(window + 1.0)
    alpha_rev = 1-alpha
    n = data.shape[0]

    pows = alpha_rev**(np.arange(n+1))

    scale_arr = 1/pows[:-1]
    offset = data[0]*pows[1:]
    pw0 = alpha*alpha_rev**(n-1)

    mult = data*pw0*scale_arr
    cumsums = mult.cumsum()
    out = offset + cumsums*scale_arr[::-1]
    return out

plt.figure()
out = numpy_ewma_vectorized_v2(np.array(running_rewards_ddpg),20)
step_list_ddpg = np.array(step_list_ddpg)

plt.plot(step_list_ddpg, out)
plt.title('Training reward over multiple runs')
plt.xlabel('Number of steps')
plt.ylabel('Cumulative reward')
plt.show()

# REINFORCE

In this section you will implement REINFORCE, with modifications for batch training. It will be for use on both discrete and continous action spaces.

## Policy Parametrization

Define a MLP which outputs a distribution over the action preferences given input state. For the discrete case, the MLP outputs the likelihood of each action (softmax) while for the continuous case, the output is the mean and standard deviation parametrizing the normal distribution from which the action is sampled.



In [ ]:
# ----------------------------------------------------
# Policy parametrizing model, MLP
# ----------------------------------------------------
# 1 or 2 hidden layers with a small number of units per layer (similar to DQN)
# use ReLU for hidden layer activations
# softmax as activation for output if discrete actions, linear for continuous control
# for the continuous case, output_dim=2*act_dim (each act_dim gets a mean and std_dev)

class mlp(nn.Module):
 

Define a function that samples an action from the policy distribtion parameters obtained as output of the MLP. The function should return the action and the log-probability (log_odds) of taking that action. 

In [ ]:
def sample_action(logit, discrete):
    # logit is the output of the softmax/linear layer
    # discrete is a flag for the environment type
    # Hint: use Categorical and Normal from torch.distributions to sample action and get the log-probability
    # Note that log_probability in this case translates to ln(\pi(a|s)) 
           
    return action, log_odds


Create a function update_policy that defines the loss function and updates the MLP according to the REINFORCE update rule (ref. slide 24 of Lec 7 or page 330 of Sutton and Barto (2018)). The update algorithm to be used below is slightly different: instead of updating the network at every time-step, we take the gradient of the loss averaged over a batch of timesteps (this is to make SGD more stable). We also use a baseline to reduce variance. 

The discount factor is set as 1 here. 


In [ ]:
def update_policy(paths, net):
    # paths: a list of paths (complete episodes, used to calculate return at each time step)
    # net: MLP object

    
    num_paths = len(paths)
    rew_cums = []
    log_odds = []
    
    for path in paths:
         # rew_cums should record return at each time step for each path 
         
         # log_odds should record log_odds obtained at each timestep of path
         
         # calculated as "reward to go"
            
    rew_cums = (rew_cums - rew_cums.mean()) / (rew_cums.std() + 1e-5) # create baseline
    
    # make log_odds, rew_cums each a vector
        
    # calculate policy loss and average over paths
    
    # take optimizer step
   

Set up environment and instantiate objects. Your algorithm is to be tested on one discrete and two continuous environments. 



In [ ]:
# Select Environment

#discrete environment:
#env_name='CartPole-v0'

#continous environments:
#env_name='InvertedPendulum-v1'
#env_name = 'HalfCheetah-v1'

env_name='InvertedPendulum-v1'

# Make the gym environment
env = gym.make(env_name)
visualize = False
animate=visualize

learning_rate = 1e-3

max_path_length=None


# Set random seeds
seed=0
torch.manual_seed(seed)
np.random.seed(seed)

# Saving parameters
logdir='./REINFORCE/'


if visualize:
    if not os.path.exists(logdir):
        os.mkdir(logdir)
    env = wrappers.Monitor(env, logdir, force=True, video_callable=lambda episode_id: episode_id%animate_interval==0)
env._max_episodes_steps = min_timesteps_per_batch


# Is this env continuous, or discrete?
discrete = isinstance(env.action_space, gym.spaces.Discrete)

# Get observation and action space dimensions
obs_dim = env.observation_space.shape[0]
act_dim = env.action_space.n if discrete else env.action_space.shape[0]

# Maximum length for episodes
max_path_length = max_path_length or env.spec.max_episode_steps

# Make network object (remember to pass in appropriate flags for the type of action space in use)
# net = mlp(*args)

# Make optimizer
optimizer = torch.optim.Adam(net.parameters(), lr = learning_rate)

## Run REINFORCE

Run REINFORCE for CartPole, InvertedPendulum, and HalfCheetah. 

In [ ]:
n_iter = 1000 
min_timesteps_per_batch = 2000  # sets the batch size for updating network
avg_reward = 0
avg_rewards = []
step_list_reinforce = []
total_steps = 0
episodes = 0

for itr in range(n_iter): # loop for number of optimization steps
    paths = []
    steps = 0
    
    while True: # loop to get enough timesteps in this batch --> if episode ends this loop will restart till steps reaches limit
        ob = env.reset()
        obs, acs, rews, log_odds = [], [], [], [] 
       
        while True: # loop for episode inside batch
            if animate_this_episode:
                env.render()
                time.sleep(0.05)
            
            # get parametrized policy distribution from net using current state ob
            
            # sample action and get log-probability (log_odds) from distribution
            
            # step environment, record reward, next state
            
            # append to obs, acs, rewards, log_odds
            
            # if done, restart episode till min_timesteps_per_batch is reached
                     
            steps += 1
            
            if done:
                episodes = episodes + 1
                break
                
        path = {"observation" : obs, 
                "reward" : np.array(rews), 
                "action" : (acs),
                "log_odds" : log_odds}
        
        paths.append(path)
        
        if steps > min_timesteps_per_batch:
            break 
        
    update_policy(paths, net)  # use all complete episodes (a batch of timesteps) recorded in this itr to update net
    
    if itr == 0:
        avg_reward = path['reward'].sum()
    else:
        avg_reward = avg_reward * 0.95 + 0.05 * path['reward'].sum()
    
    if avg_reward > 300:
        break
    
    total_steps += steps
    avg_rewards.append(avg_reward)
    step_list_reinforce.append(total_steps)
    if itr % logging_interval == 0:
        print('Average reward: {}'.format(avg_reward))
   
      
env.close()

plt.plot(avg_rewards)
plt.title('Training reward for <env> over multiple runs ')
plt.xlabel('Iteration')
plt.ylabel('Average reward')

# BONUS (15% extra)

Compare average returns for CartPole (discrete action space) when using REINFORCE and DQN. Since in REINFORCE we update the network after a set number of steps instead of after every episode, plot the average rewards as a function of steps rather than episodes for both DQN and REINFORCE. You will need to make minor edits to your DQN code from the previous assignment to record average returns as a function of time_steps.

Similarly, compare REINFORCE with DDPG on InvertedPendulum and HalfCheetah using steps for the x-axis.

You may use the example code provided below as a reference for the graphs.


In [ ]:
 # import your DQN and format your average returns as defined above

In [ ]:
plt.figure()

out = numpy_ewma_vectorized_v2(np.array(running_rewards_ddpg),20)
plt.plot(step_list_ddpg, out) # or plt.plot(step_list_DQN, out)
plt.title('Training reward over multiple runs')
plt.xlabel('Number of steps')
plt.ylabel('Cumulative reward')
plt.legend(['DDPG', 'REINFORCE']) #or plt.legend(['DQN', 'REINFORCE'])
plt.plot(step_list_reinforce, avg_rewards)

plt.show()